In [ ]:
# test that all packages load correctly
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import imageio
from sklearn.feature_extraction import image

print("🎉 all packages loaded successfully!")
print(f"📦 opencv version: {cv2.__version__}")
print(f"🔢 numpy version: {np.__version__}")
print("✅ ready to process nimslo images!")


# nimslo auto-aligning gif processor

welcome to the nimslo magic! this notebook takes your developed nimslo film shots and creates perfectly aligned gifs using some computer vision wizardry.

## what we're doing:
- load and select 4-6 images from your nimslo batch
- use a simple gui to pick reference points for alignment
- leverage cnn-based image alignment 
- match histograms for consistent exposure
- export smooth aligned gifs


In [ ]:
# imports and setup
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.widgets import Button, RectangleSelector
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# for gif creation
from PIL import Image as PILImage
import imageio

# for deep learning alignment (we'll use opencv's built-in features first, then upgrade to CNN if needed)
from sklearn.feature_extraction import image
from scipy import ndimage
from scipy.spatial.distance import cdist

# jupyter display magic
from IPython.display import display, HTML, Image as IPImage
import ipywidgets as widgets

print("🚀 all imports loaded successfully!")
print("📦 make sure you have: opencv-python, pillow, matplotlib, scikit-learn, imageio")
print("💡 tip: install with `pip install opencv-python pillow matplotlib scikit-learn imageio ipywidgets`")


In [ ]:
class NimsloProcessor:
    """
    main class for processing nimslo images into aligned gifs
    handles image loading, alignment, histogram matching, and gif export
    """
    
    def __init__(self):
        self.images = []
        self.image_paths = []
        self.reference_points = []
        self.aligned_images = []
        self.matched_images = []
        self.crop_box = None
        
    def load_images(self, folder_path=None):
        """load images from folder or file dialog"""
        if folder_path is None:
            root = tk.Tk()
            root.withdraw()
            folder_path = filedialog.askdirectory(title="select nimslo batch folder")
            root.destroy()
        
        if not folder_path:
            print("❌ no folder selected")
            return False
            
        # look for common image formats
        extensions = ['*.jpg', '*.jpeg', '*.png', '*.tiff', '*.tif']
        image_files = []
        
        for ext in extensions:
            image_files.extend(glob.glob(os.path.join(folder_path, ext)))
            image_files.extend(glob.glob(os.path.join(folder_path, ext.upper())))
        
        image_files.sort()  # ensure consistent ordering
        
        if len(image_files) < 4:
            print(f"❌ need at least 4 images, found {len(image_files)}")
            return False
            
        print(f"📁 found {len(image_files)} images")
        
        # load first 6 images max (nimslo typically has 4)
        selected_files = image_files[:6]
        self.image_paths = selected_files
        self.images = []
        
        for i, path in enumerate(selected_files):
            img = cv2.imread(path)
            if img is not None:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                self.images.append(img_rgb)
                print(f"✅ loaded image {i+1}: {os.path.basename(path)} ({img_rgb.shape})")
            else:
                print(f"❌ failed to load: {path}")
        
        return len(self.images) >= 4
    
    def show_images(self, images=None, title="nimslo batch"):
        """display loaded images in a grid"""
        if images is None:
            images = self.images
            
        if not images:
            print("❌ no images to display")
            return
            
        n_images = len(images)
        cols = min(4, n_images)
        rows = (n_images + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 4*rows))
        fig.suptitle(title, fontsize=16)
        
        if rows == 1:
            axes = [axes] if cols == 1 else axes
        else:
            axes = axes.flatten()
            
        for i, img in enumerate(images):
            axes[i].imshow(img)
            axes[i].set_title(f"frame {i+1}")
            axes[i].axis('off')
            
        # hide unused subplots
        for i in range(n_images, len(axes)):
            axes[i].axis('off')
            
        plt.tight_layout()
        plt.show()

# create processor instance
processor = NimsloProcessor()
print("🔧 nimslo processor initialized!")


In [ ]:
class InteractiveCropper:
    """interactive gui for selecting crop area and reference points"""
    
    def __init__(self, image):
        self.image = image
        self.crop_coords = None
        self.reference_points = []
        self.fig = None
        self.ax = None
        self.selector = None
        
    def on_crop_select(self, eclick, erelease):
        """callback for crop selection"""
        x1, y1 = int(eclick.xdata), int(eclick.ydata)
        x2, y2 = int(erelease.xdata), int(erelease.ydata)
        
        # ensure proper ordering
        x1, x2 = min(x1, x2), max(x1, x2)
        y1, y2 = min(y1, y2), max(y1, y2)
        
        self.crop_coords = (x1, y1, x2, y2)
        print(f"📐 crop selected: ({x1}, {y1}) to ({x2}, {y2})")
        
    def on_click(self, event):
        """callback for reference point selection"""
        if event.inaxes != self.ax:
            return
            
        if event.button == 1 and event.dblclick:  # double click
            x, y = int(event.xdata), int(event.ydata)
            self.reference_points.append((x, y))
            
            # plot the point
            self.ax.plot(x, y, 'ro', markersize=8, markeredgecolor='white', markeredgewidth=2)
            self.fig.canvas.draw()
            
            print(f"📍 reference point {len(self.reference_points)}: ({x}, {y})")
            
    def select_crop_and_reference(self):
        """interactive selection of crop area and reference point"""
        print("🎯 crop selection mode:")
        print("   - drag to select crop area")
        print("   - double-click to add reference points")
        print("   - close window when done")
        
        self.fig, self.ax = plt.subplots(figsize=(12, 8))
        self.ax.imshow(self.image)
        self.ax.set_title("select crop area (drag) and reference points (double-click)")
        
        # rectangle selector for cropping
        self.selector = RectangleSelector(
            self.ax, self.on_crop_select,
            useblit=True, button=[1], minspanx=50, minspany=50,
            spancoords='pixels', interactive=True
        )
        
        # click handler for reference points
        self.fig.canvas.mpl_connect('button_press_event', self.on_click)
        
        plt.show()
        
        return self.crop_coords, self.reference_points

def add_crop_selection_to_processor():
    """add cropping functionality to the main processor"""
    
    def select_crop_and_reference(self, image_index=0):
        """select crop area and reference points on specified image"""
        if not self.images:
            print("❌ no images loaded")
            return False
            
        if image_index >= len(self.images):
            print(f"❌ image index {image_index} out of range")
            return False
            
        cropper = InteractiveCropper(self.images[image_index])
        crop_coords, ref_points = cropper.select_crop_and_reference()
        
        if crop_coords:
            self.crop_box = crop_coords
            print(f"✅ crop area saved: {crop_coords}")
        
        if ref_points:
            self.reference_points = ref_points
            print(f"✅ {len(ref_points)} reference points saved")
            
        return crop_coords is not None or ref_points
    
    # add method to processor class
    NimsloProcessor.select_crop_and_reference = select_crop_and_reference

# add the functionality
add_crop_selection_to_processor()
print("🎨 interactive cropping functionality added!")


In [ ]:
class ImageAligner:
    """handles the alignment of images using computer vision techniques"""
    
    def __init__(self):
        # initialize feature detectors (try SIFT first, fall back to ORB)
        try:
            self.sift = cv2.SIFT_create()
            self.feature_detector = 'sift'
            print("🔍 using SIFT feature detector")
        except:
            self.sift = cv2.ORB_create(nfeatures=1000)
            self.feature_detector = 'orb'
            print("🔍 using ORB feature detector (SIFT not available)")
            
        # matcher
        if self.feature_detector == 'sift':
            self.matcher = cv2.FlannBasedMatcher()
        else:
            self.matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    def detect_and_match_features(self, img1, img2, ratio_threshold=0.7):
        """detect features and find matches between two images"""
        # convert to grayscale
        gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
        gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
        
        # detect keypoints and descriptors
        kp1, des1 = self.sift.detectAndCompute(gray1, None)
        kp2, des2 = self.sift.detectAndCompute(gray2, None)
        
        if des1 is None or des2 is None:
            print("❌ no features detected")
            return [], [], []
            
        # match features
        if self.feature_detector == 'sift':
            matches = self.matcher.knnMatch(des1, des2, k=2)
            # apply ratio test
            good_matches = []
            for match_pair in matches:
                if len(match_pair) == 2:
                    m, n = match_pair
                    if m.distance < ratio_threshold * n.distance:
                        good_matches.append(m)
        else:
            matches = self.matcher.match(des1, des2)
            good_matches = sorted(matches, key=lambda x: x.distance)[:50]
        
        return kp1, kp2, good_matches
    
    def estimate_transform(self, kp1, kp2, matches, transform_type='homography'):
        """estimate transformation matrix from matched keypoints"""
        if len(matches) < 4:
            print(f"❌ not enough matches ({len(matches)}) for transformation")
            return None
            
        # extract matched points
        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        
        if transform_type == 'homography':
            matrix, mask = cv2.findHomography(src_pts, dst_pts, 
                                            cv2.RANSAC, 5.0)
        elif transform_type == 'affine':
            matrix, mask = cv2.estimateAffinePartial2D(src_pts, dst_pts)
        else:
            print(f"❌ unsupported transform type: {transform_type}")
            return None
            
        if matrix is None:
            print("❌ failed to estimate transformation")
            return None
            
        # count inliers
        inliers = np.sum(mask) if mask is not None else len(matches)
        print(f"✅ transformation estimated with {inliers}/{len(matches)} inliers")
        
        return matrix
    
    def align_to_reference(self, images, reference_index=0, transform_type='homography'):
        """align all images to a reference image"""
        if not images or len(images) < 2:
            print("❌ need at least 2 images for alignment")
            return []
            
        reference_img = images[reference_index]
        aligned_images = [reference_img.copy()]  # reference stays unchanged
        transforms = [np.eye(3)]  # identity for reference
        
        print(f"🎯 aligning {len(images)} images to reference (image {reference_index})")
        
        for i, img in enumerate(images):
            if i == reference_index:
                continue
                
            print(f"🔄 aligning image {i}...")
            
            # detect and match features
            kp_ref, kp_img, matches = self.detect_and_match_features(reference_img, img)
            
            if len(matches) < 10:
                print(f"⚠️  warning: only {len(matches)} matches found for image {i}")
            
            # estimate transformation
            transform_matrix = self.estimate_transform(kp_ref, kp_img, matches, transform_type)
            
            if transform_matrix is not None:
                # apply transformation
                h, w = reference_img.shape[:2]
                if transform_type == 'homography':
                    aligned_img = cv2.warpPerspective(img, transform_matrix, (w, h))
                else:
                    aligned_img = cv2.warpAffine(img, transform_matrix, (w, h))
                    
                aligned_images.insert(i, aligned_img)
                transforms.insert(i, transform_matrix)
                print(f"✅ image {i} aligned successfully")
            else:
                print(f"❌ failed to align image {i}, using original")
                aligned_images.insert(i, img.copy())
                transforms.insert(i, np.eye(3))
        
        return aligned_images, transforms

# add alignment functionality to processor
def add_alignment_to_processor():
    """add alignment functionality to the main processor"""
    
    def align_images(self, reference_index=0, transform_type='homography'):
        """align all loaded images"""
        if not self.images:
            print("❌ no images loaded")
            return False
            
        aligner = ImageAligner()
        
        # apply crop if selected
        images_to_align = self.images
        if self.crop_box:
            x1, y1, x2, y2 = self.crop_box
            images_to_align = [img[y1:y2, x1:x2] for img in self.images]
            print(f"🔄 applying crop ({x1},{y1}) to ({x2},{y2})")
        
        aligned_imgs, transforms = aligner.align_to_reference(
            images_to_align, reference_index, transform_type
        )
        
        if aligned_imgs:
            self.aligned_images = aligned_imgs
            self.transforms = transforms
            print(f"✅ aligned {len(aligned_imgs)} images successfully!")
            return True
        
        return False
    
    # add method to processor
    NimsloProcessor.align_images = align_images

add_alignment_to_processor()
print("🧩 image alignment functionality added!")


In [ ]:
class HistogramMatcher:
    """handles histogram matching for consistent exposure across images"""
    
    def __init__(self, reference_index=0):
        self.reference_index = reference_index
    
    def match_histogram(self, source, reference, multichannel=True):
        """match histogram of source image to reference image"""
        if multichannel:
            # process each channel separately for color images
            matched = np.zeros_like(source)
            for channel in range(source.shape[2]):
                matched[:, :, channel] = self._match_histogram_single_channel(
                    source[:, :, channel], reference[:, :, channel]
                )
            return matched
        else:
            return self._match_histogram_single_channel(source, reference)
    
    def _match_histogram_single_channel(self, source, reference):
        """match histogram for a single channel"""
        # get histograms
        source_hist, source_bins = np.histogram(source.flatten(), 256, density=True)
        ref_hist, ref_bins = np.histogram(reference.flatten(), 256, density=True)
        
        # calculate cumulative distribution functions
        source_cdf = source_hist.cumsum()
        ref_cdf = ref_hist.cumsum()
        
        # normalize cdfs
        source_cdf = source_cdf / source_cdf[-1]
        ref_cdf = ref_cdf / ref_cdf[-1]
        
        # create lookup table
        lookup_table = np.interp(source_cdf, ref_cdf, np.arange(256))
        
        # apply lookup table
        matched = np.interp(source.flatten(), np.arange(256), lookup_table)
        
        return matched.reshape(source.shape).astype(source.dtype)
    
    def adaptive_histogram_match(self, source, reference, strength=0.7):
        """gentler histogram matching that preserves some original character"""
        matched = self.match_histogram(source, reference)
        
        # blend original with matched version
        result = (1 - strength) * source + strength * matched
        
        return result.astype(source.dtype)
    
    def match_exposure_stats(self, images, reference_index=0):
        """match basic exposure statistics (mean, std) across images"""
        if not images:
            return []
            
        reference = images[reference_index].astype(np.float32)
        ref_mean = np.mean(reference)
        ref_std = np.std(reference)
        
        matched_images = []
        
        for i, img in enumerate(images):
            if i == reference_index:
                matched_images.append(img)
                continue
                
            img_float = img.astype(np.float32)
            img_mean = np.mean(img_float)
            img_std = np.std(img_float)
            
            # normalize and rescale
            normalized = (img_float - img_mean) / (img_std + 1e-8)
            rescaled = normalized * ref_std + ref_mean
            
            # clip to valid range
            rescaled = np.clip(rescaled, 0, 255).astype(np.uint8)
            matched_images.append(rescaled)
            
        return matched_images
    
    def visualize_histograms(self, images, titles=None):
        """visualize histograms of all images for comparison"""
        n_images = len(images)
        fig, axes = plt.subplots(2, n_images, figsize=(4*n_images, 8))
        
        if titles is None:
            titles = [f"image {i+1}" for i in range(n_images)]
        
        colors = ['red', 'green', 'blue']
        
        for i, (img, title) in enumerate(zip(images, titles)):
            # show image
            axes[0, i].imshow(img)
            axes[0, i].set_title(title)
            axes[0, i].axis('off')
            
            # show histogram
            for channel, color in enumerate(colors):
                hist, bins = np.histogram(img[:, :, channel], bins=256, range=(0, 256))
                axes[1, i].plot(bins[:-1], hist, color=color, alpha=0.7, label=color)
            
            axes[1, i].set_title(f"histogram - {title}")
            axes[1, i].set_xlim(0, 255)
            axes[1, i].legend()
            
        plt.tight_layout()
        plt.show()

# add histogram matching to processor
def add_histogram_matching_to_processor():
    """add histogram matching functionality to the main processor"""
    
    def match_histograms(self, reference_index=0, method='adaptive', strength=0.7):
        """match histograms of all images"""
        if not self.aligned_images:
            print("❌ no aligned images available - run alignment first")
            return False
            
        matcher = HistogramMatcher(reference_index)
        
        print(f"🌈 matching histograms using {method} method...")
        
        if method == 'adaptive':
            matched_images = []
            reference = self.aligned_images[reference_index]
            
            for i, img in enumerate(self.aligned_images):
                if i == reference_index:
                    matched_images.append(img)
                    print(f"📊 image {i}: reference (unchanged)")
                else:
                    matched = matcher.adaptive_histogram_match(img, reference, strength)
                    matched_images.append(matched)
                    print(f"📊 image {i}: histogram matched (strength={strength})")
                    
        elif method == 'full':
            matched_images = []
            reference = self.aligned_images[reference_index]
            
            for i, img in enumerate(self.aligned_images):
                if i == reference_index:
                    matched_images.append(img)
                else:
                    matched = matcher.match_histogram(img, reference)
                    matched_images.append(matched)
                    print(f"📊 image {i}: full histogram matched")
                    
        elif method == 'exposure':
            matched_images = matcher.match_exposure_stats(self.aligned_images, reference_index)
            print("📊 exposure statistics matched")
            
        else:
            print(f"❌ unknown method: {method}")
            return False
        
        self.matched_images = matched_images
        print(f"✅ histogram matching complete!")
        return True
    
    def show_histogram_comparison(self):
        """show before/after histogram comparison"""
        if not self.aligned_images or not self.matched_images:
            print("❌ need both aligned and matched images")
            return
            
        matcher = HistogramMatcher()
        
        print("📊 before histogram matching:")
        matcher.visualize_histograms(self.aligned_images, 
                                    [f"aligned {i+1}" for i in range(len(self.aligned_images))])
        
        print("📊 after histogram matching:")
        matcher.visualize_histograms(self.matched_images,
                                   [f"matched {i+1}" for i in range(len(self.matched_images))])
    
    # add methods to processor
    NimsloProcessor.match_histograms = match_histograms
    NimsloProcessor.show_histogram_comparison = show_histogram_comparison

add_histogram_matching_to_processor()
print("🎨 histogram matching functionality added!")


In [ ]:
class GifExporter:
    """handles creation and export of animated gifs"""
    
    def __init__(self):
        self.output_folder = "nimslo_gifs"
        os.makedirs(self.output_folder, exist_ok=True)
    
    def create_gif(self, images, output_path, duration=0.2, loop=0, optimize=True):
        """create animated gif from list of images"""
        if not images:
            print("❌ no images to create gif from")
            return False
            
        print(f"🎬 creating gif with {len(images)} frames...")
        
        # convert numpy arrays to PIL images
        pil_images = []
        for i, img in enumerate(images):
            if isinstance(img, np.ndarray):
                # ensure uint8 format
                if img.dtype != np.uint8:
                    img = np.clip(img, 0, 255).astype(np.uint8)
                pil_img = PILImage.fromarray(img)
            else:
                pil_img = img
            pil_images.append(pil_img)
            print(f"✅ frame {i+1} converted")
        
        # save as gif
        try:
            pil_images[0].save(
                output_path,
                save_all=True,
                append_images=pil_images[1:],
                duration=int(duration * 1000),  # convert to milliseconds
                loop=loop,
                optimize=optimize
            )
            print(f"🎉 gif saved to: {output_path}")
            return True
        except Exception as e:
            print(f"❌ failed to save gif: {e}")
            return False
    
    def create_comparison_gif(self, original_images, processed_images, output_path, 
                            duration=0.5, show_original_first=True):
        """create a comparison gif showing original vs processed"""
        if len(original_images) != len(processed_images):
            print("❌ mismatch in number of original vs processed images")
            return False
        
        comparison_frames = []
        
        for orig, proc in zip(original_images, processed_images):
            if show_original_first:
                comparison_frames.extend([orig, proc])
            else:
                comparison_frames.extend([proc, orig])
        
        return self.create_gif(comparison_frames, output_path, duration)
    
    def create_side_by_side_gif(self, images, output_path, duration=0.2):
        """create gif showing all frames side by side"""
        if not images:
            return False
            
        # calculate grid layout
        n_images = len(images)
        cols = min(4, n_images)
        rows = (n_images + cols - 1) // cols
        
        # get dimensions
        h, w = images[0].shape[:2]
        grid_h = h * rows
        grid_w = w * cols
        
        # create grid image
        grid_img = np.zeros((grid_h, grid_w, 3), dtype=np.uint8)
        
        for i, img in enumerate(images):
            row = i // cols
            col = i % cols
            y1, y2 = row * h, (row + 1) * h
            x1, x2 = col * w, (col + 1) * w
            grid_img[y1:y2, x1:x2] = img
        
        # create single frame gif (static comparison)
        return self.create_gif([grid_img], output_path, duration=1.0)
    
    def preview_gif_frames(self, images, title="gif preview"):
        """preview frames that will be in the gif"""
        if not images:
            print("❌ no images to preview")
            return
            
        n_images = len(images)
        cols = min(4, n_images)
        rows = (n_images + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 4*rows))
        fig.suptitle(f"{title} - {n_images} frames", fontsize=16)
        
        if rows == 1 and cols == 1:
            axes = [axes]
        elif rows == 1:
            axes = [axes]
        else:
            axes = axes.flatten()
            
        for i, img in enumerate(images):
            axes[i].imshow(img)
            axes[i].set_title(f"frame {i+1}")
            axes[i].axis('off')
            
        # hide unused subplots
        for i in range(n_images, len(axes)):
            axes[i].axis('off')
            
        plt.tight_layout()
        plt.show()

# add gif export functionality to processor
def add_gif_export_to_processor():
    """add gif export functionality to the main processor"""
    
    def create_nimslo_gif(self, output_filename=None, duration=0.15, preview=True):
        """create the final nimslo gif"""
        if not self.matched_images:
            print("❌ no processed images available - run full pipeline first")
            return False
            
        exporter = GifExporter()
        
        if output_filename is None:
            timestamp = __import__('datetime').datetime.now().strftime("%Y%m%d_%H%M%S")
            output_filename = f"nimslo_{timestamp}.gif"
        
        output_path = os.path.join(exporter.output_folder, output_filename)
        
        if preview:
            print("🎬 preview of final gif frames:")
            exporter.preview_gif_frames(self.matched_images, "final nimslo gif")
        
        success = exporter.create_gif(self.matched_images, output_path, duration)
        
        if success:
            # show file info
            file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
            print(f"📁 file size: {file_size:.2f} MB")
            print(f"🎯 frames: {len(self.matched_images)}")
            print(f"⏱️  duration per frame: {duration}s")
            
            # display the gif in jupyter (if possible)
            try:
                from IPython.display import Image as IPImage
                display(IPImage(output_path))
            except:
                print("💡 gif created but couldn't display inline - check output folder")
        
        return success
    
    def create_comparison_gif(self, output_filename=None, duration=0.3):
        """create a before/after comparison gif"""
        if not self.images or not self.matched_images:
            print("❌ need both original and processed images")
            return False
            
        exporter = GifExporter()
        
        if output_filename is None:
            timestamp = __import__('datetime').datetime.now().strftime("%Y%m%d_%H%M%S")
            output_filename = f"nimslo_comparison_{timestamp}.gif"
        
        output_path = os.path.join(exporter.output_folder, output_filename)
        
        # use cropped originals if crop was applied
        original_images = self.images
        if self.crop_box:
            x1, y1, x2, y2 = self.crop_box
            original_images = [img[y1:y2, x1:x2] for img in self.images]
        
        return exporter.create_comparison_gif(
            original_images, self.matched_images, output_path, duration
        )
    
    def set_output_folder(self, folder_path):
        """set custom output folder for gifs"""
        os.makedirs(folder_path, exist_ok=True)
        GifExporter().output_folder = folder_path
        print(f"📁 output folder set to: {folder_path}")
    
    # add methods to processor
    NimsloProcessor.create_nimslo_gif = create_nimslo_gif
    NimsloProcessor.create_comparison_gif = create_comparison_gif
    NimsloProcessor.set_output_folder = set_output_folder

add_gif_export_to_processor()
print("🎬 gif export functionality added!")


## 🚀 usage guide

now that we've built all the components, here's how to use the nimslo processor:

### basic workflow:
1. **load images**: `processor.load_images()` - select your nimslo batch folder
2. **preview**: `processor.show_images()` - see what we're working with
3. **crop & reference**: `processor.select_crop_and_reference()` - interactive gui selection
4. **align**: `processor.align_images()` - computer vision magic
5. **match histograms**: `processor.match_histograms()` - consistent exposure
6. **create gif**: `processor.create_nimslo_gif()` - final output!

### quick start:
- run the cell below for an automated pipeline
- or run each step individually for more control
- all methods have helpful print statements so you know what's happening


In [ ]:
# 🎬 DEMO TIME! 
# uncomment and run the lines below to process your nimslo batch

# OPTION 1: fully automated pipeline
# process_nimslo_batch(processor)

# OPTION 2: step by step (more control)
# processor.load_images()  # select your folder
# processor.show_images()  # preview loaded images
# processor.select_crop_and_reference()  # interactive gui
# processor.align_images()  # computer vision alignment
# processor.show_images(processor.aligned_images, "aligned")
# processor.match_histograms(method='adaptive', strength=0.7)
# processor.show_histogram_comparison()
# processor.create_nimslo_gif(duration=0.15)

# OPTION 3: quick test with different settings
# processor.align_images(transform_type='affine')  # try affine vs homography
# processor.match_histograms(method='exposure')    # try different matching methods
# processor.create_nimslo_gif(duration=0.1)       # faster gif

print("🎯 ready to process! uncomment the lines above to start")
print("📁 make sure you have your nimslo images in a folder")
print("💡 tip: the interactive cropping opens a matplotlib window")
